Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
#from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28, 1), (200000, 10))
('Validation set', (10000, 28, 28, 1), (10000, 10))
('Test set', (10000, 28, 28, 1), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
 
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros([depth]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.zeros([num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.281109
Minibatch accuracy: 12.5%
Validation accuracy: 13.2%
Minibatch loss at step 50: 1.532037
Minibatch accuracy: 50.0%
Validation accuracy: 56.6%
Minibatch loss at step 100: 1.191923
Minibatch accuracy: 68.8%
Validation accuracy: 72.0%
Minibatch loss at step 150: 0.646593
Minibatch accuracy: 81.2%
Validation accuracy: 75.8%
Minibatch loss at step 200: 0.643753
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 250: 0.620779
Minibatch accuracy: 93.8%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.348903
Minibatch accuracy: 93.8%
Validation accuracy: 80.4%
Minibatch loss at step 350: 0.282127
Minibatch accuracy: 87.5%
Validation accuracy: 79.5%
Minibatch loss at step 400: 0.496490
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 450: 0.967681
Minibatch accuracy: 68.8%
Validation accuracy: 81.0%
Minibatch loss at step 500: 1.244859
Minibatch accuracy: 56.2%
Validation accuracy: 81.5%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [61]:
def feedforward_train(data, c_1_w,c_1_b,  c_2_w,c_2_b,   f_1_w,f_1_b,   f_2_w,f_2_b):
    c_1 = tf.nn.relu(tf.nn.conv2d(data,c_1_w,[1,2,2,1],padding="SAME") + c_1_b)
    c_1 = tf.nn.max_pool(c_1,[1,2,2,1],[1,2,2,1],padding = "SAME")
    c_2 = tf.nn.relu(tf.nn.conv2d(c_1,c_2_w,[1,2,2,1],padding="SAME") + c_2_b)
    c_2 = tf.nn.max_pool(c_2,[1,2,2,1],[1,2,2,1],padding = "SAME")
    shape = c_2.get_shape().as_list()
    reshaped_c_2 = tf.reshape(c_2,[shape[0],shape[1]*shape[2]*shape[3]])
    f_1 = tf.nn.relu(tf.matmul(reshaped_c_2,f_1_w) + f_1_b)
    d_f_1 = tf.nn.dropout(f_1,0.2)
    f_2 = tf.matmul(d_f_1,f_2_w) + f_2_b
    return f_2

def feedforward(data, c_1_w,c_1_b,  c_2_w,c_2_b,   f_1_w,f_1_b,   f_2_w,f_2_b):
    c_1 = tf.nn.relu(tf.nn.conv2d(data,c_1_w,[1,2,2,1],padding="SAME") + c_1_b)
    c_1 = tf.nn.max_pool(c_1,[1,2,2,1],[1,2,2,1],padding = "SAME")
    c_2 = tf.nn.relu(tf.nn.conv2d(c_1,c_2_w,[1,2,2,1],padding="SAME") + c_2_b)
    c_2 = tf.nn.max_pool(c_2,[1,2,2,1],[1,2,2,1],padding = "SAME")
    shape = c_2.get_shape().as_list()
    reshaped_c_2 = tf.reshape(c_2,[shape[0],shape[1]*shape[2]*shape[3]])
    f_1 = tf.nn.relu(tf.matmul(reshaped_c_2,f_1_w) + f_1_b)
    f_2 = tf.matmul(f_1,f_2_w) + f_2_b
    return f_2

In [62]:
batch_size = 200
depth = 20
image_size = 28
patch_size = 5
hidden_1_size = 80


g1 = tf.Graph()
with g1.as_default():
    tf_train_data = tf.placeholder(tf.float32,shape=(batch_size,image_size,image_size,num_channels))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_val_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    c_weights1 = tf.Variable(tf.truncated_normal([patch_size,patch_size,num_channels,depth],stddev=0.1))
    c_biases1 = tf.Variable(tf.zeros([depth]))
    
    c_weights2 = tf.Variable(tf.truncated_normal([patch_size,patch_size,depth,depth],stddev=0.1))
    c_biases2 = tf.Variable(tf.zeros([depth]))
    
    f_weights1 = tf.Variable(tf.truncated_normal([image_size//12*image_size//12*depth,hidden_1_size],stddev= 0.1))
    f_biases1 = tf.Variable(tf.zeros([hidden_1_size]))
    
    f_weights2 = tf.Variable(tf.truncated_normal([hidden_1_size,num_labels], stddev=0.1))
    f_biases2 = tf.zeros([num_labels])
    
    p = [c_weights1,c_biases1,c_weights2,c_biases2,f_weights1,f_biases1,f_weights2,f_biases2]
    
    logits = feedforward_train(tf_train_data,p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    val_prediction = tf.nn.softmax(feedforward(tf_val_dataset,p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7]))
    test_prediction = tf.nn.softmax(feedforward(tf_test_dataset,p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7]))

In [63]:
num_steps = 10001

with tf.Session(graph = g1) as sess:
    sess.run(tf.initialize_all_variables())
    
    for i in range(num_steps):
        start = (i*batch_size)% (len(train_labels) - batch_size)
        end = start + batch_size
        train_subset = train_dataset[start:end,:,:,:]
        label_subset = train_labels[start:end,:]
        feed_d = {tf_train_data:train_subset,tf_train_labels:label_subset}
        
        _,l = sess.run([optimizer,loss],feed_dict=feed_d)
        
        if i % 2000 == 0:
            print "batch_loss at iteration -", i," = ",l
            print "valid_accuracy", accuracy(val_prediction.eval(),valid_labels)
    print "test_accuracy",accuracy(test_prediction.eval(),test_labels)

batch_loss at iteration - 0  =  2.31922
valid_accuracy 14.62
batch_loss at iteration - 2000  =  0.723891
valid_accuracy 84.9
batch_loss at iteration - 4000  =  0.62987
valid_accuracy 86.24
batch_loss at iteration - 6000  =  0.660464
valid_accuracy 86.98
batch_loss at iteration - 8000  =  0.58948
valid_accuracy 87.29
batch_loss at iteration - 10000  =  0.623905
valid_accuracy 87.6
test_accuracy 93.87


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---